In [4]:
import numpy as np
import pandas as pa
letters_train = pa.read_csv(
    'https://www.labri.fr/perso/zemmari/datasets/emnist/emnist-letters-train.csv',
    header=None)
letters_test = pa.read_csv(
    'https://www.labri.fr/perso/zemmari/datasets/emnist/emnist-letters-test.csv',
    header=None)
def load_letters():
    X_train = np.array(letters_train.iloc[:, 1:785])
    y_train = np.array(letters_train.iloc[:, 0])
    y_train = y_train - 1
    X_test = np.array(letters_test.iloc[:, 1:785])
    y_test = np.array(letters_test.iloc[:, 0])
    y_test = y_test -1 
    X_train = np.reshape(X_train, (X_train.shape[0], 28, 28))
    X_test = np.reshape(X_test, (X_test.shape[0], 28, 28))
    return (X_train, y_train), (X_test, y_test)

# Partie 1

In [6]:
# Question 1
import tensorflow as tf
import keras
from keras.utils import to_categorical
(X_train, y_train), (X_test, y_test) = load_letters()
img_rows, img_cols = X_train.shape[1], X_train.shape[2]

X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train  = X_train / 255
X_test  = X_test / 255

unique_classes = np.unique(y_train)
print(unique_classes)

num_classes = unique_classes.size


y_train = to_categorical(y_train,num_classes)
y_test = to_categorical(y_test,num_classes)
print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_test shape: {y_test.shape}")

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25]
X_train shape: (88800, 28, 28, 1)
y_train shape: (88800, 26)
X_test shape: (14800, 28, 28, 1)
y_test shape: (14800, 26)


In [7]:
# Question 2
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

def letter_network():
    model = Sequential()
    model.add(Conv2D(32,kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64,kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(26, activation='softmax')) #26 lettres dans l'alphabet
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

model = letter_network()
model.summary()
history = model.fit(X_train, y_train, epochs=10, batch_size=100, validation_data=(X_test, y_test))
scores = model.evaluate(X_test, y_test)
print("Test loss:", scores[0])
print("Test accuracy:", scores[1])

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 13, 13, 32)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 1600)              0         
                                                                 
 dense_2 (Dense)             (None, 128)              

# Partie 2

In [18]:
# Question 1
from keras.datasets import mnist
# load des lettres
(X_train_letters, y_train_letters), (X_test_letters, y_test_letters) = load_letters()

# load des chiffres
(X_train_numbers, y_train_numbers), (X_test_numbers, y_test_numbers) = mnist.load_data()

# on concatene les données
X_train = np.concatenate((X_train_letters,X_train_numbers))
y_train = np.concatenate((y_train_letters,y_train_numbers))

X_test = np.concatenate((X_test_letters,X_test_numbers))
y_test = np.concatenate((y_test_letters,y_test_numbers))


# on definit la taille
img_rows, img_cols = X_train.shape[1], X_train.shape[2]

# On reshape en 1D
X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)

# on transforme y_train et y_test en vecteurs
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_test shape: {y_test.shape}")

X_train shape: (148800, 28, 28, 1)
y_train shape: (148800, 26)
X_test shape: (24800, 28, 28, 1)
y_test shape: (24800, 19)


In [16]:
def neural_network():
    model = Sequential()
    model.add(Conv2D(32,kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)))
    model.add(MaxPooling2D((2,2)))
    model.add(Conv2D(64,kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(2, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

model = neural_network()
model.summary()
history = model.fit(X_train, y_train, epochs=10, batch_size=100, validation_data=(X_test, y_test))
scores = model.evaluate(X_test, y_test)
print("Test loss:", scores[0])
print("Test accuracy:", scores[1])

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 13, 13, 32)       0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 flatten_3 (Flatten)         (None, 1600)              0         
                                                                 
 dense_6 (Dense)             (None, 128)              

ValueError: in user code:

    File "D:\Anaconda\lib\site-packages\keras\engine\training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "D:\Anaconda\lib\site-packages\keras\engine\training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "D:\Anaconda\lib\site-packages\keras\engine\training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "D:\Anaconda\lib\site-packages\keras\engine\training.py", line 1024, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "D:\Anaconda\lib\site-packages\keras\engine\training.py", line 1082, in compute_loss
        return self.compiled_loss(
    File "D:\Anaconda\lib\site-packages\keras\engine\compile_utils.py", line 265, in __call__
        loss_value = loss_obj(y_t, y_p, sample_weight=sw)
    File "D:\Anaconda\lib\site-packages\keras\losses.py", line 152, in __call__
        